# Imports

In [2]:
from sklearn import datasets
import pandas as pd
import numpy as np
import os 
import seaborn as sns
from sklearn import svm
from sklearn import metrics

from sklearn.model_selection import train_test_split

# encoders
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv('df_balanced_full.csv', low_memory=False)

In [4]:
display(df)

,value_chartevent,valuenum_chartevent,valueuom_chartevent,label_chartevent,category,time_since_admission_chartevent,icd_code,admission_type,insurance,race,admission_location,age,value_labevent,valuenum_labevent,valueuom_labevent,priority,time_since_admission_labevent
0,103,103.0,°F,Temperature Fahrenheit,Routine Vital Signs,4.850000,2724,EW EMER.,Medicaid,UNABLE TO OBTAIN,EMERGENCY ROOM,60,-3,-3.00,mEq/L,Unknown,12.316667
1,25,25.0,mEq/L,TCO2 (calc) Arterial,Labs,7.900000,2724,SURGICAL SAME DAY ADMISSION,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,1.4,1.40,Unknown,STAT,4.150000
2,25,25.0,mA,Temporary Ventricular Stim Setting mA,Cardiovascular (Pacer Data),16.750000,2724,SURGICAL SAME DAY ADMISSION,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,384,384.00,mm Hg,Unknown,5.616667
3,36.8,36.8,°C,Temperature Celsius,Routine Vital Signs,18.750000,2724,SURGICAL SAME DAY ADMISSION,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,98,98.00,%,Unknown,6.666667
4,100,100.0,%,O2 saturation pulseoxymetry,Respiratory,22.750000,2724,ELECTIVE,Medicare,WHITE,PHYSICIAN REFERRAL,70,1.2,1.20,Unknown,ROUTINE,15.366667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37195,0 Alert and calm,0.0,Unknown,Goal Richmond-RAS Scale,Pain/Sedation,3.583333,Z87891,EW EMER.,Other,WHITE,EMERGENCY ROOM,70,___,36.00,mm Hg,Unknown,4.133333
37196,127,127.0,bpm,Heart Rate,Routine Vital Signs,21.800000,Z87891,URGENT,Medicare,UNKNOWN,TRANSFER FROM HOSPITAL,81,38.2,38.20,%,ROUTINE,18.600000
37197,475,475.0,mL,Tidal Volume (observed),Respiratory,23.516667,Z87891,OBSERVATION ADMIT,Medicare,WHITE,TRANSFER FROM HOSPITAL,58,71,71.00,fL,STAT,15.233333
37198,12,12.0,insp/min,Respiratory Rate,Respiratory,11.916667,Z87891,SURGICAL SAME DAY ADMISSION,Medicare,WHITE,PHYSICIAN REFERRAL,72,7.37,7.37,units,Unknown,16.300000


In [5]:

print("Features: ", df.columns.tolist())
print("Labels: ", df['icd_code'].unique())


Features:  ['value_chartevent', 'valuenum_chartevent', 'valueuom_chartevent', 'label_chartevent', 'category', 'time_since_admission_chartevent', 'icd_code', 'admission_type', 'insurance', 'race', 'admission_location', 'age', 'value_labevent', 'valuenum_labevent', 'valueuom_labevent', 'priority', 'time_since_admission_labevent']
Labels:  ['2724' '4019' 'E039' 'E785' 'Z794' 'Z87891']


In [6]:
df.head()

,value_chartevent,valuenum_chartevent,valueuom_chartevent,label_chartevent,category,time_since_admission_chartevent,icd_code,admission_type,insurance,race,admission_location,age,value_labevent,valuenum_labevent,valueuom_labevent,priority,time_since_admission_labevent
0,103,103.0,°F,Temperature Fahrenheit,Routine Vital Signs,4.85,2724,EW EMER.,Medicaid,UNABLE TO OBTAIN,EMERGENCY ROOM,60,-3,-3.0,mEq/L,Unknown,12.316667
1,25,25.0,mEq/L,TCO2 (calc) Arterial,Labs,7.90,2724,SURGICAL SAME DAY ADMISSION,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,1.4,1.4,Unknown,STAT,4.150000
2,25,25.0,mA,Temporary Ventricular Stim Setting mA,Cardiovascular (Pacer Data),16.75,2724,SURGICAL SAME DAY ADMISSION,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,384,384.0,mm Hg,Unknown,5.616667
3,36.8,36.8,°C,Temperature Celsius,Routine Vital Signs,18.75,2724,SURGICAL SAME DAY ADMISSION,Other,HISPANIC OR LATINO,PHYSICIAN REFERRAL,67,98,98.0,%,Unknown,6.666667
4,100,100.0,%,O2 saturation pulseoxymetry,Respiratory,22.75,2724,ELECTIVE,Medicare,WHITE,PHYSICIAN REFERRAL,70,1.2,1.2,Unknown,ROUTINE,15.366667


# Treatment Strings

In [7]:
# Strings with medium number of unique types -> Frequency
variables = [
   'value_chartevent', 'label_chartevent', 'valueuom_labevent', 'valueuom_chartevent'
]

# String with the higest number of unique types -> Frequency
lab = ['value_labevent']


# Strings with low number of unique types -> OneHotEncoding
variables_cat = ['admission_type', 'insurance', 'priority', 'race', 'admission_location', 'category']

# Numerical
numerical_cols = [
    'valuenum_chartevent', 'valuenum_labevent',
    'time_since_admission_chartevent', 'time_since_admission_labevent', 'age'
]

# Target (icd_code)
target = 'icd_code'

In [8]:
X_raw = df[variables + numerical_cols + variables_cat + lab].copy()
y = df[target].copy()

In [9]:
le = LabelEncoder()
y_encoded = le.fit_transform(y)

y = y_encoded

In [10]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
one_hot_encoded = encoder.fit_transform(X_raw[variables_cat])

one_hot_df = pd.DataFrame(one_hot_encoded, 
                          columns=encoder.get_feature_names_out(variables_cat),
                          index=X_raw.index)

X_encoded = pd.concat([X_raw.drop(columns=variables_cat), one_hot_df], axis=1)

In [11]:
for col in variables:
    freq = X_encoded[col].value_counts()
    X_encoded[col + '_freq'] = X_encoded[col].map(freq)

# Remove original high unique values columns
X_encoded.drop(columns=variables, inplace=True)

# missing values indicator

for col in numerical_cols:
    X_encoded[col + '_missing'] = X_encoded[col].isna().astype(int)

# Preencher os NaNs com um valor neutro (-999)
X_encoded[numerical_cols] = X_encoded[numerical_cols].fillna(-999)

display(X_encoded)

,valuenum_chartevent,valuenum_labevent,time_since_admission_chartevent,time_since_admission_labevent,age,value_labevent,admission_type_DIRECT EMER.,admission_type_ELECTIVE,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,...,category_Treatments,value_chartevent_freq,label_chartevent_freq,valueuom_labevent_freq,valueuom_chartevent_freq,valuenum_chartevent_missing,valuenum_labevent_missing,time_since_admission_chartevent_missing,time_since_admission_labevent_missing,age_missing
0,103.0,-3.00,4.850000,12.316667,60,-3,0.0,0.0,1.0,0.0,...,0.0,131,474,8052,474,0,0,0,0,0
1,25.0,1.40,7.900000,4.150000,67,1.4,0.0,0.0,0.0,0.0,...,0.0,147,121,2204,1559,0,0,0,0,0
2,25.0,384.00,16.750000,5.616667,67,384,0.0,0.0,0.0,0.0,...,0.0,147,7,2052,57,0,0,0,0,0
3,36.8,98.00,18.750000,6.666667,67,98,0.0,0.0,0.0,0.0,...,0.0,17,124,4113,212,0,0,0,0,0
4,100.0,1.20,22.750000,15.366667,70,1.2,0.0,1.0,0.0,0.0,...,0.0,914,1863,2204,3052,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37195,0.0,36.00,3.583333,4.133333,70,___,0.0,0.0,1.0,0.0,...,0.0,385,294,2052,13476,0,0,0,0,0
37196,127.0,38.20,21.800000,18.600000,81,38.2,0.0,0.0,0.0,0.0,...,0.0,27,1903,4113,2307,0,0,0,0,0
37197,475.0,71.00,23.516667,15.233333,58,71,0.0,0.0,0.0,1.0,...,0.0,2,113,1270,351,0,0,0,0,0
37198,12.0,7.37,11.916667,16.300000,72,7.37,0.0,0.0,0.0,0.0,...,0.0,243,1846,1232,2751,0,0,0,0,0


In [12]:

X_encoded['value_labevent'] = X_encoded['value_labevent'].astype(str)

freq = X_encoded['value_labevent'].value_counts()
X_encoded['value_labevent_freq'] = X_encoded['value_labevent'].map(freq)

# Remove original
X_encoded.drop(columns=['value_labevent'], inplace=True)

In [13]:
X = X_encoded

# Nan

In [14]:
X = X.dropna()

# Model

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [16]:
cols = X_train.columns

In [17]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [18]:
X_train = pd.DataFrame(X_train, columns=[cols])

In [19]:
X_test = pd.DataFrame(X_test, columns=[cols])

In [20]:
X_train.describe()

,valuenum_chartevent,valuenum_labevent,time_since_admission_chartevent,time_since_admission_labevent,age,admission_type_DIRECT EMER.,admission_type_ELECTIVE,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,...,value_chartevent_freq,label_chartevent_freq,valueuom_labevent_freq,valueuom_chartevent_freq,valuenum_chartevent_missing,valuenum_labevent_missing,time_since_admission_chartevent_missing,time_since_admission_labevent_missing,age_missing,value_labevent_freq
count,2.976000e+04,2.976000e+04,2.976000e+04,2.976000e+04,2.976000e+04,2.976000e+04,2.976000e+04,2.976000e+04,2.976000e+04,2.976000e+04,...,2.976000e+04,2.976000e+04,2.976000e+04,2.976000e+04,29760.0,29760.0,29760.0,29760.0,29760.0,2.976000e+04
mean,2.984470e-18,-1.910061e-17,-3.132500e-16,1.656978e-16,-2.043765e-16,-4.440892e-17,-5.467550e-17,1.069634e-16,-7.401487e-17,-1.957813e-17,...,2.244322e-17,-3.981284e-17,-3.748495e-17,-8.022257e-17,0.0,0.0,0.0,0.0,0.0,-3.151601e-17
std,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,...,1.000017e+00,1.000017e+00,1.000017e+00,1.000017e+00,0.0,0.0,0.0,0.0,0.0,1.000017e+00
min,-1.281597e-01,-8.971949e-02,-1.939575e+00,-3.666185e+00,-2.343028e+00,-2.011226e-01,-2.081632e-01,-1.038650e+00,-4.511031e-01,-3.769187e-01,...,-6.028174e-01,-8.931661e-01,-1.492978e+00,-1.391844e+00,0.0,0.0,0.0,0.0,0.0,-4.351847e-01
25%,-9.220701e-02,-7.219809e-02,-8.835067e-01,-6.493871e-01,-4.653846e-01,-2.011226e-01,-2.081632e-01,-1.038650e+00,-4.511031e-01,-3.769187e-01,...,-5.151752e-01,-6.807061e-01,-9.897470e-01,-9.512866e-01,0.0,0.0,0.0,0.0,0.0,-4.035749e-01
50%,-6.896052e-02,-6.428520e-02,-3.385179e-02,-3.971305e-03,2.141185e-02,-2.011226e-01,-2.081632e-01,9.627886e-01,-4.511031e-01,-3.769187e-01,...,-4.241621e-01,-5.390661e-01,-7.015558e-02,8.973616e-02,0.0,0.0,0.0,0.0,0.0,-3.508919e-01
75%,2.489177e-02,-4.097043e-02,8.284040e-01,7.726096e-01,6.472930e-01,-2.011226e-01,-2.081632e-01,9.627886e-01,-4.511031e-01,-3.769187e-01,...,-2.185400e-01,3.453971e-01,9.736862e-01,1.182533e+00,0.0,0.0,0.0,0.0,0.0,-2.464839e-01
max,9.037288e+01,4.419209e+01,1.775865e+00,1.865652e+00,1.968598e+00,4.972092e+00,4.803922e+00,9.627886e-01,2.216788e+00,2.653092e+00,...,2.674776e+00,2.100159e+00,1.293138e+00,1.182533e+00,0.0,0.0,0.0,0.0,0.0,3.026565e+00


In [21]:
clf = svm.SVC()

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))
